# Code documentation Q&A bot example with LangChain

This Q&A bot will allow you to query your own documentation easily using questions. We'll also demonstrate the use of LangChain and LanceDB using the OpenAI API. 

In this example we'll use Pandas 2.0 documentation, but, this could be replaced for your own docs as well

In [ ]:
!pip install -qq openai langchain tiktoken
!pip install -qq lancedb

First, let's get some setup out of the way. As we're using the OpenAI API, ensure that you've set your key (and organization if needed):

In [1]:
import openai
import os

# Configuring the environment variable OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = "sk-..."
if "OPENAI_API_KEY" not in os.environ:
    # OR set the key here as a variable
    os.environ["OPENAI_API_KEY"] = "sk-..."
    
assert len(openai.Model.list()["data"]) > 0

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

In [2]:
import lancedb
db = lancedb.connect("data/sample-lancedb")
db.open_table("sup")

LanceTable(sup)

# Loading in our code documentation, generating embeddings and storing our documents in LanceDB

We're going to use the power of LangChain to help us create our Q&A bot. It comes with several APIs that can make our development much easier as well as a LanceDB integration for vectorstore.

In [3]:
import lancedb
import re
import pickle
import requests
import zipfile
from pathlib import Path

from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import LanceDB
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

To make this easier, we've downloaded Pandas documentation and stored the raw HTML files for you to download. We'll download them and then use LangChain's HTML document readers to parse them and store them in LanceDB as a vector store, along with relevant metadata.

In [4]:
pandas_docs = requests.get("https://eto-public.s3.us-west-2.amazonaws.com/datasets/pandas_docs/pandas.documentation.zip")
with open('/tmp/pandas.documentation.zip', 'wb') as f:
    f.write(pandas_docs.content)

file = zipfile.ZipFile("/tmp/pandas.documentation.zip")
file.extractall(path="/tmp/pandas_docs")

We'll create a simple helper function that can help to extract metadata, so we can use this downstream when we're wanting to query with filters. In this case, we want to keep the lineage of the uri or path for each document that we process:

In [5]:
def get_document_title(document):
    m = str(document.metadata["source"])
    title = re.findall("pandas.documentation(.*).html", m)
    if title[0] is not None:
        return(title[0])
    return ''

# Pre-processing and loading the documentation

Next, let's pre-process and load the documentation. To make sure we don't need to do this repeatedly if we were updating code, we're caching it using pickle so we can retrieve it again (this could take a few minutes to run the first time you do it). We'll also add some more metadata to the docs here such as the title and version of the code:

In [6]:
docs_path = Path("docs.pkl")
docs = []

if not docs_path.exists():
    for p in Path("/tmp/pandas_docs/pandas.documentation").rglob("*.html"):
        print(p)
        if p.is_dir():
            continue
        try:
            loader = UnstructuredHTMLLoader(p)
            raw_document = loader.load()
        except Exception as e:
            print(f"Error loading document: {e}")
            continue
        
        m = {}
        m["title"] = get_document_title(raw_document[0])
        m["version"] = "2.0rc0"
        raw_document[0].metadata = raw_document[0].metadata | m
        raw_document[0].metadata["source"] = str(raw_document[0].metadata["source"])
        docs = docs + raw_document

    with docs_path.open("wb") as fh:
        pickle.dump(docs, fh)
else:
    with docs_path.open("rb") as fh:
        docs = pickle.load(fh)


# Generating emebeddings from our docs

Now that we have our raw documents loaded, we need to pre-process them to generate embeddings:

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()

# Storing and querying with LanceDB

Let's connect to LanceDB so we can store our documents. We'll create a Table to store them in:

In [8]:
db = lancedb.connect('lancedb')
table = db.create_table("pandas_docs", data=[
    {"vector": embeddings.embed_query("Hello World"), "text": "Hello World", "id": "1"}
], mode="overwrite")
docsearch = LanceDB.from_documents(documents, embeddings, connection=table)

Skipped index 6626 due to embedding error
Skipped index 8487 due to embedding error
Skipped index 8575 due to embedding error
Skipped index 10230 due to embedding error


Now let's create our RetrievalQA chain using the LanceDB vector store:

In [9]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever())

And thats it! We're all setup. The next step is to run some queries, let's try a few:

In [10]:
query = "What are the major differences in pandas 2.0?"
qa.run(query)

' Enhancements such as installing optional dependencies with pip extras, the ability to use any numpy numeric dtype in a Index, and several bug fixes. Additionally, there are certain backwards incompatible API changes, deprecations, and performance improvements.'

In [11]:
query = "What's the current version of pandas?"
qa.run(query)

' 2.0.0rc0'

In [12]:
query = "How do I make use of installing optional dependencies?"
qa.run(query)

' You can install optional dependencies with the pip command, using the extra syntax (e.g. ``pandas[performance, aws]>=1.5.0``). Additionally, you can use mamba or pyenv to create a virtual environment for your development environment, and then install the build dependencies with the ``pip install -r requirements-dev.txt`` command. Finally, you should use the private helper ``pandas.compat._optional.import_optional_dependency`` to import any optional dependencies, along with adding a test to make sure an ImportError is raised if the dependency is not found.'

In [13]:
query = "What are the backwards incompatible API changes in Pandas 2.0?"
qa.run(query)

' Possible incompatibility for HDF5 formats created with pandas < 0.13.0, Map on Index types now return other Index types, Accessing datetime fields of Index now return Index, pd.unique will now be consistent with extension types, S3 file handling, Partial string indexing changes, Concat of different float dtypes will not automatically upcast, pandas Google BigQuery support has moved, Memory usage for Index is more accurate, DataFrame.sort_index changes, GroupBy describe formatting, Window binary corr/cov operations return a MultiIndex DataFrame, HDFStore where string comparison, Dependencies have increased minimum versions, Sum/prod of all-NaN or empty Series/DataFrames is now consistently NaN, Indexing with a list with missing labels is deprecated, NA naming changes, Iteration of Series/Index will now return Python scalars, Indexing with a Boolean Index, PeriodIndex resampling, Improved error handling during item assignment in pd.eval, Dtype conversions, MultiIndex constructor with a